# Mesa Schelling example - Schelling Segregation Model

[[Code explanation]](https://towardsdatascience.com/introduction-to-mesa-agent-based-modeling-in-python-bcb0596e1c9a) **Note that the final interactive visualization part we will cover in the later parts of the course**

## Background

The Schelling (1971) segregation model is a classic of agent-based modeling, demonstrating how agents following simple rules lead to the emergence of qualitatively different macro-level outcomes. Agents are randomly placed on a grid. There are two types of agents, one constituting the majority and the other the minority. All agents want a certain number (generally, 3) of their 8 surrounding neighbors to be of the same type in order for them to be happy. Unhappy agents will move to a random available grid space. While individual agents do not have a preference for a segregated outcome (e.g. they would be happy with 3 similar neighbors and 5 different ones), the aggregate outcome is nevertheless heavily segregated.

In [ ]:
import random

from mesa import Model, Agent
from mesa.time import RandomActivation
from mesa.space import SingleGrid
from mesa.datacollection import DataCollector

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Agent

class SchellingAgent(Agent):
    '''
    Schelling segregation agent
    '''
    
    # 1 Initialization
    def __init__(self, pos, model, agent_type):
        '''
         Create a new Schelling agent.

         Args:
            unique_id: Unique identifier for the agent.
            x, y: Agent initial location.
            agent_type: Indicator for the agent's type (minority=1, majority=0)
        '''
        super().__init__(pos, model)
        self.pos = pos
        self.type = agent_type

    # 2 Step function
    def step(self):
    
        n_neighbors = len(self.model.grid.get_neighbors(self.pos, True))

        # Is no neighbors - agent happy
        if n_neighbors == 0:
            self.model.happy += 1
            return
        
        similar_neighbors = 0
        neighbors = self.model.grid.neighbor_iter(self.pos)
            
        # 3 Calculate the number of similar neighbours
        for neighbor in neighbors:
            if neighbor.type == self.type:
                similar_neighbors += 1
                        
        # Is Agent already happy with it's location
        loc_happines = (similar_neighbors/n_neighbors) * 100
        if loc_happines >= self.model.homophily:
            self.model.happy += 1
            return
            
        # if Unhappy move to better empty cell
        empty_happines_list = {}
        for empty in self.model.grid.empties:
            
            n_empty_neighbors = len(self.model.grid.get_neighbors(empty, True))
            
            # Is no neighbors - agent happy
            if n_empty_neighbors == 0:
                self.model.grid.move_agent(self, empty)
                return
            
            similar = 0
            empty_neighbors = self.model.grid.neighbor_iter(empty)
            
            for neighbor in empty_neighbors:
                if neighbor.type == self.type:
                    similar += 1
                    
            # 4 Move to a random empty location if unhappy
            empty_loc_happines = (similar / n_empty_neighbors) * 100
            #if empty_loc_happines >= self.model.homophily:
            #    self.model.grid.move_agent(self, empty)
            #    return
            
            empty_happines_list[empty] = empty_loc_happines
            
        # Move to a maximal better place from the list of empties
        max_happines_empty_loc = max(empty_happines_list, key=empty_happines_list.get)
        
        if empty_happines_list[max_happines_empty_loc] >= loc_happines:
            self.model.grid.move_agent(self, max_happines_empty_loc)

In [ ]:
# Data collection

def get_morans_i(model):
    '''
    Find Moran's I for all agents and neighbours on the grid.
    '''
    
    x_kappa = model.minority_pc
    N = model.schedule.get_agent_count()
    edges_var = 0
    edges_sum = 0
    total_edges = 0
    
    for agent in model.schedule.agents:
        edges_var += (agent.type - x_kappa) ** 2
        for neighbor in model.grid.neighbor_iter(agent.pos):
            total_edges += 1
            edges_sum += (agent.type - x_kappa) * (neighbor.type - x_kappa)
            
    return (N * edges_sum) / (total_edges * edges_var)

def get_segregation_edges(model):
    '''
    Find the % of agents that only have neighbors of their same type.
    '''
    segregated_edges = 0
    total_edges = 0
    for agent in model.schedule.agents:
        for neighbor in model.grid.neighbor_iter(agent.pos):
            total_edges += 1
            if neighbor.type == agent.type:
                segregated_edges += 1
    return segregated_edges / total_edges

def get_segregation(model):
    '''
    Find the % of agents that only have neighbors of their same type.
    '''
    segregated_agents = 0
    for agent in model.schedule.agents:
        segregated = True
        for neighbor in model.grid.neighbor_iter(agent.pos):
            if neighbor.type != agent.type:
                segregated = False
                break
        if segregated:
            segregated_agents += 1
    return segregated_agents / model.schedule.get_agent_count()

In [ ]:
# Model

class SchellingModel(Model):
    '''
    Model class for the Schelling segregation model.
    '''

    def __init__(self, height, width, density, minority_pc, homophily):
        '''
        Create a new Schelling model.

         Args:
            width: Horizontal axis of the grid which is used together with Height to define the total number of agents in the system.
            height: Vertical axis of the grid which is used together with Width to define the total number of agents in the system.
            density: Define the population density of agent in the system. Floating value from 0 to 1.
            fraction minority: The ratio between blue and red. Blue is represented as the minority while red is represented as the majority. Floating value from 0 to 1. If the value is higher than 0.5, blue will become the majority instead.
            homophily: Define the number of similar neighbors required for the agents to be happy. Integer value range from 0 to 8 since you can only be surrounded by 8 neighbors.
        '''
        super().__init__()
        self.height = height
        self.width = width
        self.density = density
        self.minority_pc = minority_pc
        self.homophily = homophily

        self.schedule = RandomActivation(self)
        self.grid = SingleGrid(height, width, torus=True)

        self.happy = 0
        self.datacollector = DataCollector(
            {"Happy": "happy", "Segregated_Agents": get_segregation, "Segregated_Edges": get_segregation_edges, "Morans_I": get_morans_i},  # Model-level count of happy agents
            # For testing purposes, agent's individual x and y
            {"x": lambda a: a.pos[0], "y": lambda a: a.pos[1]})

        self.running = True

        # Set up agents
        # We use a grid iterator that returns
        # the coordinates of a cell as well as
        # its contents. (coord_iter)
        for cell in self.grid.coord_iter():
            x = cell[1]
            y = cell[2]
            if random.random() < self.density:
                if random.random() < self.minority_pc:
                    agent_type = 1
                else:
                    agent_type = 0

                agent = SchellingAgent((x, y), self, agent_type)
                self.grid.position_agent(agent, (x, y))
                self.schedule.add(agent)

    def step(self):
        '''
        Run one step of the model. If All agents are happy, halt the model.
        '''
        
        # Spontaneous move 1%
        spontaneous_agents_count = 0
        shuffled_agents = self.schedule.agent_buffer(shuffled = True)
        for agent in shuffled_agents:
            if spontaneous_agents_count / self.schedule.get_agent_count() <= 0.01:
                agent.spontaneous_move = True
                spontaneous_agents_count += 1
            else:
                break
            
        self.happy = 0  # 1 Reset counter of happy agents
        self.schedule.step()
        
        # 2 collect data
        self.datacollector.collect(self)
        
        # 3 Stop the model if all agents are happy
        if self.happy == self.schedule.get_agent_count():
            self.running = False

Now we instantiate a model instance: a 10x10 grid, with an 80% chance of an agent being placed in each cell, approximately 20% of agents set as minorities, and agents wanting at least 3 similar neighbors.

In [ ]:
model = SchellingModel(50, 50, 0.98, 0.5, 60)

We want to run the model until all the agents are happy with where they are. However, there's no guarentee that a given model instantiation will *ever* settle down. So let's run it for either 100 steps or until it stops on its own, whichever comes first:

In [ ]:
while model.running and model.schedule.steps < 100:
    model.step()

print('The Schelling Model ran for {} steps'.format(model.schedule.steps))

# Visualization of data

In [ ]:
from mesa.visualization.modules import CanvasGrid
from mesa.visualization.ModularVisualization import ModularServer


def agent_portrayal(agent):
    portrayal = {"Shape": "circle",
                 "Filled": "true",
                 "Layer": 0,
                 "r": 0.5}
    
    if agent.type == 0:
        portrayal["Color"] = "Red"
    else:
        portrayal["Color"] = "Blue"

    return portrayal

grid = CanvasGrid(agent_portrayal, 20, 20, 500, 500)
server = ModularServer(SchellingModel,
                       [grid],
                       "Schelling Model",
                       {"density":0.98, "width":20, "height":20, "minority_pc":0.5, "homophily":20})
server.port = 8540 # The default
server.launch()

The model has a DataCollector object, which checks and stores how many agents are happy at the end of each step. It can also generate a pandas DataFrame of the data it has collected:

In [ ]:
model_out = model.datacollector.get_model_vars_dataframe()
model_out.head()

Finally, we can plot the 'happy' series:

In [ ]:
model_out.Happy.plot()

For testing purposes, here is a table giving each agent's x and y values at each step.

In [ ]:
x_positions = model.datacollector.get_agent_vars_dataframe()
x_positions.head()

# Effect of Homophily on segregation

Now, we can do a parameter sweep to see how segregation changes with homophily.

First, we create a function which takes a model instance and returns what fraction of agents are segregated -- that is, have no neighbors of the opposite type.

In [ ]:
from mesa.batchrunner import batch_run

params = {"height": 20, "width": 20, "density": 0.98, "minority_pc": 0.5, 
              "homophily": range(0,100, 13)}

results = batch_run(
    SchellingModel,
    parameters=params,
    iterations=10,
    max_steps=200,
    number_processes=1,
    display_progress=True,
)

### Complete data is available for each model instantiation for each agent for each step

In [ ]:
import pandas as pd

results_df = pd.DataFrame(results)
print(results_df.keys())

In [ ]:
results_df.boxplot(by ='homophily', column =['Segregated_Agents'], grid=False)

In [ ]:
results_df.groupby(by=["RunId"]).median().boxplot(by ='homophily', column =['Segregated_Agents'], grid=False)

In [ ]:
results_df.boxplot(by ='homophily', column =['Segregated_Edges'], grid=False)

In [ ]:
results_df.groupby(by=["RunId"]).median().boxplot(by ='homophily', column =['Segregated_Edges'], grid=False)

In [ ]:
results_df.groupby(by=["RunId"]).median().boxplot(by ='homophily', column =['Morans_I'], grid=False)
plt.suptitle(None)
plt.title(None)
plt.ylabel("Moran's I")
plt.savefig('morans_i_plot.eps')